# **Importação das Bibliotecas**

In [1]:
import polars as pl
from datetime import datetime

# **Leitura dos Dados**

In [2]:
raw_data = pl.read_csv("../data/01_raw/store_item_demand.csv")
raw_data.head()

date,store,item,sales
str,i64,i64,i64
"""2013-01-01""",1,1,13
"""2013-01-02""",1,1,11
"""2013-01-03""",1,1,14
"""2013-01-04""",1,1,13
"""2013-01-05""",1,1,10


# **Análise Inicial**

In [3]:
raw_data.describe()

statistic,date,store,item,sales
str,str,f64,f64,f64
"""count""","""913000""",913000.0,913000.0,913000.0
"""null_count""","""0""",0.0,0.0,0.0
"""mean""",null,5.5,25.5,52.250287
"""std""",null,2.872283,14.430878,28.801144
"""min""","""2013-01-01""",1.0,1.0,0.0
"""25%""",null,3.0,13.0,30.0
"""50%""",null,6.0,26.0,47.0
"""75%""",null,8.0,38.0,70.0
"""max""","""2017-12-31""",10.0,50.0,231.0


## **Datas**

In [4]:
gb_raw_data = raw_data.group_by(["store", "item"]).len().sort(by="len")

start = datetime.strptime(raw_data["date"].min(), "%Y-%m-%d")
end = datetime.strptime(raw_data["date"].max(), "%Y-%m-%d")
all_dates = pl.datetime_range(start=start, end=end, interval="1d", eager=True).alias("date")

equal_dates = 0
stores = raw_data["store"].unique().to_list()
items = raw_data["item"].unique().to_list()
for store in stores:
    for item in items:
        store_item_data = raw_data.filter(
            (pl.col("store") == store) & (pl.col("item") == item)
        )
        if (all_dates == store_item_data["date"].str.to_datetime().sort()).all():
            equal_dates += 1

if equal_dates == gb_raw_data.shape[0]:
    print("Todas as séries temporais possuem as mesmas datas.")
else:
    print("As séries temporais não possuem as mesmas datas.")

Todas as séries temporais possuem as mesmas datas.


## **Demandas zeradas**

In [5]:
raw_data.filter(pl.col("sales") == 0)

date,store,item,sales
str,i64,i64,i64
"""2014-01-15""",6,4,0


# **Conclusões**

- Nehuma série possui valores faltantes (NaN);
- Nenhuma série possui demanda negativa ou algum valor absurdo que precise ser tratado; 
- Todas as séries possuem dados diários de "01/01/2013" até "31/12/2017", sem faltar nenhum dia;
- Só existe 1 registro com demanda (sales) igual a 0, portanto não será necessário utilizar um modelo de previsão de demanda intermitente.